<a href="https://colab.research.google.com/github/felipefcunica/cogroo4py/blob/master/Scripts/Classes_Comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import re
import nltk
import numpy as np
import string
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
!python -m spacy download pt
import spacy
pln = spacy.load('pt')
from spacy.lang.pt.stop_words import STOP_WORDS
import pt_core_news_sm
from scipy import sparse  
from scipy.sparse import hstack
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
import regex as re
pln.Defaults.stop_words |= {"seja"," “ ","pessoa", #alterar para entrada .txt
                            "pessoas","uso", "fato",
                            "será", "muita", "visto",
                            "outro", "maneira", "d",
                            "n", "há", "seja", "muitas",
                            "fica", "si", "passar", "á",
                            "ficar", "causou", "passando",
                            "passou", "perante", "alguém",
                            "terá", "poucos", "tornou",
                            "podendo", "tenham", "ocorre",
                            "voltar", "todavia", "dando",
                            "eram", "tantos", "conseguinte",
                            "necessário", "devido","havia",
                            "feito", "tinha", "faria", "poderiam",
                            "tê", "lo", "ninguém","sendo",
                            "entretanto", "apesar", "melhor",
                            "teriam", "mesma", "colocam",
                            "haver", "s", "leva", "levar",
                            "e",'a','o'}

     |████████████████████████████████| 21.2 MB 1.3 MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-py3-none-any.whl size=21186282 sha256=96f03c27bb1f341ebd4b4de72069c2555e97878982e7004f5d2c0e4436806625
  Stored in directory: /tmp/pip-ephem-wheel-cache-0kwwlj3y/wheels/c3/f9/0c/5c014a36941a00f5df5fc0756cb961d7c457a978e697a6ce3b
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **COMPETÊNCIA 1**

Demonstrar domínio da escrita formal da língua portuguêsa.

**Última atualização em:** 27/10/2021 por *Felipe Ferreira*.

## Desvios

**1.  Desvios mais graves**

*   Falta de concordância - Sujeito antes do Verbo;
*   Períodos incompletos que comprometem a compreensão;
*   Graves problemas de pontuação;
*   Desvios graves de grafia e de acentuação( ex. nomes próprios); **✓**
*   Presença de gíria. **✓**

**2.  Desvios graves**

*   Falta de concordância - Sujeito distante ou depois do verbo;
*   Falta de concordância do adjetivo com o substantivo;
*   Ausência ou emprego indevido de preposição;
*   Ausência do acento indicativo da crase ou uso inadequado;
*   Problemas na estrutura sintática;
*   Frases sem conectivos ou orações subordinadas sem oração principal;
*   Desvios em palavras de grafia complexa; **✓**
*   Separação de sujeito, verbo, objeto direto e indireto por vírgula e **✓**
*   Marcas da oralidade. **✓**

**3.  Desvios Leves**

*   Ausência de concordância em passiva sintética(exemplo: uso de “vende-se casas” em vez de “vendem-se casas”) e
*   Desvios de pontuação que não comprometem o sentido do texto.

## Documentação

**|||||||||| Objetivo ||||||||||**

Esta classe estabelece um complexo fluxo onde uma redação é processada e analisada de acordo aos desvios listados na Competência 1 do ENEM. 

---

**|||||||||| Inicialização ||||||||||**

O método construtor desta classe recebe os seguintes argumentos:

 `refVocab:`
*   **Type:** Str ;
*   **Descrição:** Diretório para importar vocabulário geral.

`refNomesprop`
*   **Type:** Str ;
*   **Descrição :** Diretório para importar vocabulário de nomes próprios.

`refGir`
*   **Type:** Str ; 
*   **Descrição:** Diretório para importar palavras e locuções que identifiquem liguagem de baixo calão.

`refConjun`
*   **Type:** Str ;
*   **Descrição:** Diretório para importar conjunções adverbiais.

`refEmbVocab`
*   **Type:** Str ;
*   **Descrição:** Diretório para importar modelo de vocabulário vetorizado.
*   **Obs:** Vetorização de vocabulário tem de estar disponível para ser lida no modo bytes.

`refVpModel`
*   **Type:** Str ;
*   **Descrição:** Diretório para importar modelo de classificação para voz passiva.
*   **Obs:** Modelo tem de estar disponível para ser lido no modo bytes.

---


**|||||||||| Execução ||||||||||**

Para executar atribua à classe a uma variável com os devidos diretórios nos argumentos.
```
competencia1 = comp1(refVocab,refNomespropri,refGir,refConjun,refEmbVocab,refVpModel)
```
Em seguida atribua o método `analisar(redação)`, sendo redação sua variável do tipo string a ser analisada.
```
competencia1.analisar(redação)
```


---



## Classe

In [ ]:
class comp1:

  
  #______Inicialização______#

  def importData(self,data): #Subfunction apontada em __init__()
    imp=[]
    with open(os.path.join(data), "r", encoding='utf-8') as f:
      text = f.readlines()
      for palavra in text:
        imp.append(palavra.replace('\n',''))    
    return imp

  def __init__(self,refVocab,refNomespropri,refGir,refConjun,refEmbVocab,refVpModel):
    self.vocab = self.importData(refVocab)
    self.nomespropri = self.importData(refNomespropri)
    self.Gir = self.importData(refGir)
    self.conjAdv = self.importData(refConjun)
    self.pontuacoes = string.punctuation
    self.refGeru = ['estar'] #Inserir lib em .txt
    self.embVocab = pickle.load(open(refEmbVocab,'rb'))
    self.vpModel = pickle.load(open(refVpModel, 'rb'))
    

  #______END______#

  #______Gramática_e_Ortografia______#

  def corretor(self,input_txt,dicionario,nomesref): #Corretor ortográfico V2.0.
    lst_erradas = [ ]
    temp = re.sub(r'[!"#$%&\'()*+,/:;<=>?@\\^_`{|}~,`´]','',input_txt.replace('.',' ')).split()
    for palavra_errada in temp:
      count = 0
      if palavra_errada in nomesref:
        count+=1
      elif palavra_errada.lower() in dicionario:
        count+=1
      elif type(re.match(r'[0-9]',palavra_errada)) != type(None):
        count+=1
      elif count == 0:
        lst_erradas.append(palavra_errada)
    return lst_erradas

  def virg(self,ent,ref): #Uso de vírgula V3.0.
    err_lis,cord_lis = [],[]
    for cong in ref:
      a = re.search(f'{cong}',ent.lower())
      if type(a) != type(None):
        if ent[int(a.span()[0])-2] == '.':
          if ent[int(a.span()[1])] != ',':
            err_lis.append(cong)
            cord_lis.append(a.span())
        elif ent[int(a.span()[0])-1] == ' ': 
          if ent[int(a.span()[0]-2)] != ',':
            err_lis.append(cong)
            cord_lis.append(a.span())
        elif ent[int(a.span()[0])-1] == '\n': 
          if ent[int(a.span()[1])] != ',':
            err_lis.append(cong)
            cord_lis.append(a.span())
        elif int(a.span()[0]) == 0:
          if ent[int(a.span()[1])] != ',':
            err_lis.append(cong)
            cord_lis.append(a.span()) 
    return err_lis,cord_lis

  #______END______#

  #______ORALIDADE______#

  def detGiria(self,entrada): #Detecta o uso de gírias.
    giria = []        
    for palavra in self.Gir:
      if palavra in entrada:
        giria.append(palavra)
    return giria

  def detec_gerundismo(self,entrada,key): #MainFunction de detecção de gerundismo
    redal = entrada.split('.')            
    saida = []                         
    trecho = []
    loc = []
    for f in redal:
      saida.append(self.frasger(f,self.refGeru))
    for r in range(len(saida)):
      if saida[r] != []:
        trecho.append(saida[r])
        loc.append(r)
    return trecho,loc

  def frasger(self,txt,key):  #Subfunction apontada em detec_gerundismo().
    txtl = txt.lower().split()
    geru = []
    for w in range(len(txtl)):
      if txtl[w] in key:
        if (txtl[w+1])[-1] in self.pontuacoes:
          if txtl[w+1][-4:-1] == 'ndo':
            geru.append(f'{txtl[w]} {txtl[w+1]}')
        else:
          if txtl[w+1][-3:] == 'ndo':
            geru.append(f'{txtl[w]} {txtl[w+1]}')
    return geru

  def vect(self,entrada,embVocab): #vetorização da redação, subfuncition 
    vocabulario = self.embVocab #apontada em vozPassiva()
    r_split = re.split('[.]|[,]',entrada.replace('\n','')) 
    red_emb = vocabulario.transform(r_split) #vetorização
    return r_split,red_emb
  
  def vozPassiva(self,entrada,embVocab,vpModel): # MainFunction para
    modelo = self.vpModel           #detecção do uso de
    red_spl,red_vec = self.vect(entrada,self.embVocab)  #voz passiva.
    vp = modelo.predict(red_vec) #predição usando ML
    res = []
    for i in range(len(vp)):
      if vp[i] == 1:
        res.append(red_spl[i])
    return res
  #______END______#


  #______Execução______#
  def analisar(self,entrada): #Função de análise textual.
    lis_corr = self.corretor(entrada,self.vocab,self.nomespropri)
    lis_vir,cord_vir = self.virg(entrada,self.conjAdv)
    lis_giria = self.detGiria(entrada)
    lis_geru = self.detec_gerundismo(entrada,self.refGeru)
    lis_vp = self.vozPassiva(entrada,self.embVocab,self.vpModel)
    data = {'ortografia':lis_corr,
            'uso_de_virgula':lis_vir,
            'posi_virgula':cord_vir,
            'girias':lis_giria,
            'gerundismo':lis_geru,
            'voz_passiva':lis_vp
            }

    return data

  # No futuro inserir aqui uma função que atribua uma nota
  #com base nas análises contidas nesta competência.

  #______END______#

## Execução

In [ ]:
vocab = '/content/drive/MyDrive/Spell_AI/Data/vocabularios/vocab4.0.txt'
nom = '/content/drive/MyDrive/Spell_AI/Data/vocabularios/nomesprop2.0.txt'
giria = '/content/drive/MyDrive/Spell_AI/Data/vocabularios/girias.txt'
conj_adv = '/content/drive/MyDrive/Spell_AI/Data/vocabularios/conjadv.txt'
vocab_emb =  '/content/drive/MyDrive/Spell_AI/Data/modelos/vec_vozpassiva'
vp_decis = '/content/drive/MyDrive/Spell_AI/Data/modelos/decis_vozpassiva.txt'

In [ ]:
competencia1 = comp1(vocab,nom,giria,conj_adv,vocab_emb,vp_decis)

In [ ]:
ent = '''A população brasileira apontou a fé como fator importante para melhorar a vida  deixando  fatores como estudos, acesso à saúde, aposentadoria, trabalho, lazer e outros para trás. Essa foi a resposta de 28% dos entrevistados  que caracteriza o Brasil como uma sociedade com moral enraizada em preceitos e mandamentos religiosos e que isso gera melhora de vida como consequência. Esse resultado é a coadunação do senso comum com a ciência, pois são numerosas as pesquisas médicas que incluem a fé como assunto a ser tratado em consultórios, hospitais e até grades acadêmicas.
A fé colabora para a participação de um fenômeno chamado coping: a capacidade humana de superar adversidades e não é necessário crer em uma divindade, pois os estudos voltados para essa área encontram essa capacidade também em ateus que creem em máximas ou jaculatórias como "vai ficar tudo bem", "eu vou superar isso" e outras afirmações positivas, colhendo efeitos minorados de um diagnóstico de doença como menores níveis de estresse. Equivale dizer  que os benefícios da fé à saúde encontram embasamento científico.
A  exemplo disso é a Santa Casa de Porto Alegre que  em parceria com a Universidade Duke, dos Estados Unidos, tem desenvolvido estudos para mensurar os benefícios biológicos da fé. Mas há mais, as principais universidades norte americanas  têm dedicado uma disciplina específica ao assunto. Significa  que a fé influencia no trato que temos com as pessoas, nas reações diante das dificuldades do dia-a-dia  e também na reação com os acontecimentos bons.
A fé é ponto fundamental para melhorar a vida, seja a própria, seja a vida em sociedade, inclusive em tempos de crise, segregação e instabilidade político-econômica  que o brasileiro vive e esse é o porquê a população apontou a fé antes dos demais itens da pesquisa, pois assim já dizia Gilberto Gil "andar com fé eu vou que a fé não costuma falhar".'''

In [ ]:
resul = competencia1.analisar(ent)
resul

{'gerundismo': ([], []),
 'girias': [],
 'ortografia': ['coping',
  'creem',
  'minorados',
  'dia-a-dia',
  'político-econômica'],
 'posi_virgula': [(1820, 1825)],
 'uso_de_virgula': ['assim'],
 'voz_passiva': [' Essa foi a resposta de 28% dos entrevistados  que caracteriza o Brasil como uma sociedade com moral enraizada em preceitos e mandamentos religiosos e que isso gera melhora de vida como consequência',
  ' pois são numerosas as pesquisas médicas que incluem a fé como assunto a ser tratado em consultórios']}

In [ ]:
resul['ortografia']

['coping', 'creem', 'minorados', 'dia-a-dia', 'político-econômica']

In [ ]:
resul['voz_passiva']

[' Essa foi a resposta de 28% dos entrevistados  que caracteriza o Brasil como uma sociedade com moral enraizada em preceitos e mandamentos religiosos e que isso gera melhora de vida como consequência',
 ' pois são numerosas as pesquisas médicas que incluem a fé como assunto a ser tratado em consultórios']

# **COMPETÊNCIA 2**
  Compreender a proposta de redação e aplicar conceitos das várias áreas do conhecimento para desenvolver o tema dentro dos limites estruturais do texto dissertativo-argumentativo.

**Última atualização em:** 05/11/2021 por *Felipe Ferreira*.

## Desvios



*   Fuga Parcial ao tema ou não cumpriu quantidade de linhas solicitadas; **✓**
*   Fuga Total ao Tema. Tema Competamente diferente do proposto. Redação   Anulada; **✓**
*   Fuga ao Gênero: Desenvolver texto que não obedeça a estrutura dissertativo-argumentativa. Redação Anulada. **✓**






## Documentação

**|||||||||| Objetivo ||||||||||**

Completamente baseada em aprendizagem de máquinas, esta classe estabelece um fluxo de classificação relacionado ao tema e gênero do texto.

---

**|||||||||| Inicialização ||||||||||**

O método construtor desta classe recebe os seguintes argumentos:

 `refEmbVocab:`
*   **Type:** Str ;
*   **Descrição:** Diretório para importar vocabulário geral vetorizado.

`refFugaModel`
*   **Type:** Str ;
*   **Descrição :** Diretório para importar o modelo pré-treinado de classificação de fuga ao tema.

`refGloveModel`
*   **Type:** Str ; 
*   **Descrição:** Diretório para importar o modelo pré-treinado GloveS50.

`refXGBClassText`
*   **Type:** Str ; 
*   **Descrição:** Diretório para importar o modelo pré-treinado XGB para classificação de tipo textual

`refXGBVec`
*   **Type:** Str ; 
*   **Descrição:** Diretório para importar o modelo de vocabulário vetorizado para XGB.

`refPrimPessoa`
*   **Type:** Str ; 
*   **Descrição:** Diretório para importar o lista de elementos de primeira pessoa.

---


**|||||||||| Execução ||||||||||**

Para executar atribua à classe a uma variável com os devidos diretórios nos argumentos.
```
competencia2 = comp2(refEmbVocab,refFugaModel,refGloveModel,refXGBClassText,refXGBVec,refPrimPessoa)
```
Em seguida atribua o método `analisar(redação,tema,motivador)`, sendo todas variáveis do tipo string a serem analisadas.
```
competencia2.analisar(redação,tema,motivador)
```


---



## Classe

In [11]:
class comp2:

  #______Inicialização______#

  def importData(self,data): #Subfunction apontada em __init__()
    imp=[]
    with open(os.path.join(data), "r", encoding='utf-8') as f:
      text = f.readlines()
      for palavra in text:
        imp.append(palavra.replace('\n',''))    
    return imp

  def __init__(self,refEmbVocab,refFugaModel,refGloveModel,refXGBClassText,refXGBVec,refPrimPessoa):
    self.embVocab = pickle.load(open(refEmbVocab, 'rb'))
    self.fugaModel = joblib.load(refFugaModel)
    self.modeloGlove = KeyedVectors.load_word2vec_format(refGloveModel)
    self.modeloXGB = joblib.load(refXGBClassText)
    self.XGBVec = pickle.load(open(refXGBVec, 'rb'))
    self.pontuacoes = string.punctuation
    self.primPess = self.importData(refPrimPessoa)
    pln.Defaults.stop_words |= {"seja"," “ ","pessoa", #alterar para entrada .txt
                            "pessoas","uso", "fato",
                            "será", "muita", "visto",
                            "outro", "maneira", "d",
                            "n", "há", "seja", "muitas",
                            "fica", "si", "passar", "á",
                            "ficar", "causou", "passando",
                            "passou", "perante", "alguém",
                            "terá", "poucos", "tornou",
                            "podendo", "tenham", "ocorre",
                            "voltar", "todavia", "dando",
                            "eram", "tantos", "conseguinte",
                            "necessário", "devido","havia",
                            "feito", "tinha", "faria", "poderiam",
                            "tê", "lo", "ninguém","sendo",
                            "entretanto", "apesar", "melhor",
                            "teriam", "mesma", "colocam",
                            "haver", "s", "leva", "levar"  }
    self.stop_words = pln.Defaults.stop_words
  #______END______#

  #______Uso_da_Primeira_Pessoa______#
  def detecPrimeiraPessoa(self,entrada):
    primP = []
    ltxt = entrada.lower()
    for i in self.primPess:
      var = (i,[ltxt.find(i),ltxt.find(i)+len(i)])
      if ltxt.find(i) != -1:
        primP.append(var)
    return primP


  #______Fuga_ao_Tema______#

  #__Motor_de_classificação__#
  def preprocessamento(self,text): #Subfunction apontada em limpatextoetransformaemvetor()
      result = []
      pos_tag = ['ADJ', 'NOUN', 'VERB', 'PROPN'] # escolhendo apenas adjetivos, substantivos e verbos , 'VERB'
      texto = re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ: ]', '', text )
      doc = pln(texto.lower()) # Transformando em minúsculo
      for token in doc:
          # 3
          if(token.text in self.stop_words or token.text in self.pontuacoes):
              continue
          # 4
          if (token.pos_ in pos_tag):
              result.append(token.text)
                
          texto = ' '.join([str(elemento) for elemento in result if not elemento.isdigit()])         
                
      #return result # 5
      return texto # 5

  def limpatextoetransformaemvetor(self,entrada, tema, motivador):#Subfunction apontada
      limporedacao = [self.preprocessamento(entrada)]             #em previsao()
      limpotema = self.preprocessamento(tema)
      limpomotivador = self.preprocessamento(motivador)
      juntos = [limpotema+ " " +limpomotivador]
      return limporedacao, juntos

  def previsao(self, redacaolimpa, temalimpo): #subfunction apontadada em classificarFuga()
      red = self.embVocab.transform(redacaolimpa)
      mot = self.embVocab.transform(temalimpo)
      pronto = sparse.hstack((red,mot))   
      previsao = self.fugaModel.predict(pronto)
      percentual = self.fugaModel.predict_proba(pronto)[:,1]
      result = percentual[0]*100
      return previsao, result
    
  #__Similaridade_Semântica__#

  def palavrasimportates(self,entrada):
    result = []
    pos_tag = ['ADJ', 'NOUN', 'VERB', 'PROPN'] # escolhendo apenas adjetivos, substantivos e verbos , 'VERB'
    texto = re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ: ]', '', entrada )
    doc = pln(texto.lower()) # Transformando em minúsculo
    for token in doc:
        # 3
        if (token.text in self.stop_words or token.text in self.pontuacoes):
            continue
        # 4
        if (token.pos_ in pos_tag):
            result.append(token.text)
        texto = ' '.join([str(elemento) for elemento in result if not elemento.isdigit()]) 
    return texto
  
  def listar_similsent(self,motivador): #Lista termos de sentimento similar
    result_sent = []
    positive_words = [
                    palavra for palavra in self.palavrasimportates(motivador).split() 
                    if palavra in self.modeloGlove
                    ]
    result_sent.append(
      self.modeloGlove.most_similar(
          positive=positive_words,topn=10)
      )
    return result_sent
  
  def testscore (self,lst_count): #por sugestão de estudo usar fator == 5.
    similsem = []
    if 5 >= sum(lst_count):
      similsem.append(f'42,22% de probabilidade em fuga ao tema')
    if 5 <= sum(lst_count):
      similsem.append(f'57,78% de probabilidade em fuga ao tema')
    return similsem

  def relacaoproposta2(self,entrada,tema,motivador):
    texto = self.palavrasimportates(entrada)
    tem = self.palavrasimportates(tema)
    prop = self.palavrasimportates(motivador)
    palavras_spl = motivador.translate(
      motivador.maketrans('','',string.punctuation)
      ).lower().split()
    text_spl = texto.translate(
        texto.maketrans('','',string.punctuation)
      ).lower().split()
    lis_sim = self.listar_similsent(tem)
    for sim in lis_sim[0]:
      if sim[0] not in self.stop_words:
        palavras_spl.append(sim[0])
    lst_pal,lst_count,lst_ret=[],[],[] #Adaptação a partir daqui.
    for palavra in palavras_spl:
      cnt = 0
      for match in text_spl:
        if palavra == match:
          cnt+=1
          if palavra not in lst_ret:
            lst_ret.append(palavra)
        else:
          continue   
      if cnt>= 1:
        lst_pal.append(palavra)
        lst_count.append(cnt)
      else:
        continue
    similsem = sum(lst_count)
    return similsem, lst_ret

  #______ClassificaçãoTextual______#

  def classTipoText(self,text):
    sup = []
    sup.append(text)
    red = self.XGBVec.transform(sup)
    predClassText = self.modeloXGB.predict(red)
    return predClassText

  #______END______#
  
  #______Execução______#
  
  def analisar(self,entrada,tema,motivador): #Mainfunction de fuga ao tema
    limpo, unidos = self.limpatextoetransformaemvetor(entrada,tema,motivador)
    prev, res = self.previsao(limpo,unidos)
    relacaoseman,lisseman = self.relacaoproposta2(entrada,tema,motivador)
    tipoTextual = (self.classTipoText(entrada))[0]
    primPesso_res = self.detecPrimeiraPessoa(entrada) 
    data = {'predicao_fuga': prev,
            'resultado':f'{round(res,2)}%',
            'similaridade_semantica':relacaoseman,
            'lista_similaridade': lisseman,
            'tipo_textual':tipoTextual,
            'primeira_pessoa':primPesso_res}
    return data

## Execução

In [12]:
refEmbVocab = '/content/drive/MyDrive/Spell_AI/Data/modelos/Cópia de vectorizer.pickle'
refFugaModel = '/content/drive/MyDrive/Spell_AI/Data/modelos/Cópia de gradienteboosting.pkl'
refGloveModel = '/content/drive/MyDrive/Spell_AI/Data/modelos/glove_s50.txt'
refXGBClassText = '/content/drive/MyDrive/Spell_AI/Data/modelos/modeloXGBClassText'
refXGBVec = '/content/drive/MyDrive/Spell_AI/Data/modelos/vec_tipotex'
refPrimPessoa = '/content/drive/MyDrive/Spell_AI/Data/bases/primeirapessoa.txt'

In [13]:
competencia2 = comp2(refEmbVocab,refFugaModel,refGloveModel,refXGBClassText,refXGBVec,refPrimPessoa)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator DummyClassifier from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your 

In [14]:
txta='''A população brasileira apontou a fé como fator importante para melhorar a vida deixando  fatores como estudos, acesso à saúde, aposentadoria, trabalho, lazer e outros para trás. Essa foi a resposta de 28% dos entrevistados  que caracteriza o Brasil como uma sociedade com moral enraizada em preceitos e mandamentos religiosos e que isso gera melhora de vida como consequência. Esse resultado é a coadunação do senso comum com a ciência, pois são numerosas as pesquisas médicas que incluem a fé como assunto a ser tratado em consultórios, hospitais e até grades acadêmicas.
Nós jamais saberemos se a fé colabora para a participação de um fenômeno chamado coping : a capacidade humana de superar adversidades e não é necessário crer em uma divindade, pois os estudos voltados para essa área encontram essa capacidade também em ateus que creem em máximas ou jaculatórias como "vai ficar tudo bem", "eu vou superar isso" e outras afirmações positivas, colhendo efeitos minorados de um diagnóstico de doença como menores níveis de estresse. Equivale dizer  que os benefícios da fé à saúde encontram embasamento científico.
A  exemplo disso é a Santa Casa de Porto Alegre que  em parceria com a Universidade Duke, dos Estados Unidos, tem desenvolvido estudos para mensurar os benefícios biológicos da fé. Mas há mais, as principais universidades norte americanas  têm dedicado uma disciplina específica ao assunto. Significa  que a fé influencia no trato que temos com as pessoas, nas reações diante das dificuldades do dia-a-dia  e também na reação com os acontecimentos bons.
A fé é ponto fundamental para melhorar a vida, seja a própria, seja a vida em sociedade, inclusive em tempos de crise, segregação e instabilidade político-econômica  que o brasileiro vive e esse é o porquê a população apontou a fé antes dos demais itens da pesquisa, pois assim  já dizia Gilberto Gil "andar com fé eu vou que a fé não costuma falhar".'''
txtm='''Reportagem publicada pelo UOL Economia no mês passado apresenta uma pesquisa realizada pelo Datafolha em que se perguntava aos entrevistados qual o fator mais importante para se conquistar uma vida melhor. A amostragem reflete toda a população do Brasil, com baixa margem de erro. Os resultados da pesquisa revelaram que, em primeiro lugar, os brasileiros consideram necessária a fé religiosa. De acordo com a pesquisa, as pessoas consideram a fé mais importante do que o estudo ou o trabalho, por exemplo, para melhorar de vida. 
Leia o texto do UOL que se transcreve abaixo, preste atenção nos percentuais e redija uma dissertação argumentativa apresentando sua opinião sobre a questão formulada na pesquisa: para você, dos itens mencionados, qual é o mais importante para melhorar de vida? Por quê? Apresente suas razões para justificar o seu ponto de vista.Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vidaA fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. Esse percentual supera até mesmo aqueles que consideram os estudos (21%), o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor.
Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição.Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%), apoio financeiro da família (5%) e cultura e lazer (2%).Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam:- Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter acesso à aposentadoria: 6% - Apoio financeiro da família: 5% - Cultura e lazer: 2% Pesquisa ouviu 2.086 pessoas em todo o país. A Oxfam encomendou a pesquisa ao Instituto Datafolha, que entrevistou pessoalmente 2.086 pessoas em 130 cidades de todos os estados brasileiros. As entrevistas foram feitas entre os dias 12 e 18 de fevereiro. 
A margem de erro para a amostragem geral é de dois pontos percentuais para mais ou para menos, considerando um nível de confiança de 95%.Por ser uma pesquisa amostral realizada em locais de grande circulação de pessoas, o Datafolha definiu uma amostra que busca refletir o perfil da sociedade brasileira conforme o último Censo, informou a Oxfam. '''
tem = '''Como melhorar a qualidade de vida nas cidades?'''

In [15]:
tema1 =  '''Brasileiros têm "péssima educação argumentativa", segundo cientista'''
motivador1 = '''Polarização. Você já deve ter ouvido essa palavra recentemente, aplicada às opiniões de grande parte da população brasileira, em especial no que se refere a questões políticas. A polarização surge da divergência de ideias e tem gerado não somente discussões e debates, mas verdadeiros conflitos, nem sempre no âmbito exclusivamente verbal. Diante desse vale-tudo, é o caso de se perguntar: é para isso que serve o debate, a discussão, o confronto de ideias? Os brasileiros não têm uma correta educação argumentativa? Leia os textos que integram esta proposta de redação, reflita sobre o assunto e redija uma dissertação argumentativa em forma de carta, em que você vai tentar explicar ao destinatário como deve ser um debate, para que ele dê resultados, produza esclarecimento para ambas as partes e seja, afinal, construtivo. Você é capaz de argumentar sobre o modo correto de debater, evitando a polarização? Patrulha moralista Estão todos com pedras na mão prontos para mirar no próximo infiel que dê o menor sinal de pecador. 
 Uma frase mal colocada, um título com ironia ou a falta de reverência a uma causa bastam para os patrulheiros começarem o massacre moralista no Facebook. A patrulha moralista domina tanto a esquerda quanto a direita. Olavo de Carvalho leu um gibi da “Turma da Mônica” com a frase “meu corpo, minhas regras” e em milésimos de segundo concluiu: a revistinha havia se transformado “num odiento discurso abortista”! A roteirista da publicação foi ameaçada e ganhou montagens com ofensas.
Na verdade, a Mônica do gibi usou a frase para reclamar de quem insistia para ela usar aparelho dentário e deixar de ser dentuça. Na esquerda, Jean Wyllys é quem costuma entender errado declarações só para ouriçar seus seguidores e ganhar compartilhamentos. Em 2012, Gilberto Dimenstein se perguntou aqui na Folha se São Paulo precisava “importar um baiano” para a secretaria de Cultura. Bastava ler o texto para perceber que o colunista defendia a contratação do baiano Juca Ferreira porque São Paulo “é aberta, marcada pela diversidade”. Mas Jean Wyllys não quis entender: viu ali uma demonstração de xenofobia e de ódio paulista aos nordestinos.Leandro Narloch, Folha de S. Paulo  Discussão serve para construir conhecimento Não é fácil vencer uma discussão. Especialmente em um contexto inflamado, em que as opiniões se polarizam, notícias falsas se proliferam, debatedores recorrem a ofensas e sarcasmo e festas de fim de ano criam ambientes propícios para a briga. Uma boa discussão, ao contrário do que a maior parte das pessoas pensa, não serve para a disputa - e, sim, para a construção do conhecimento. Nesse sentido, saber sustentar uma boa argumentação é fundamental. 
 Walter Carnielli, matemático, professor de lógica na Unicamp (Universidade Estadual de Campinas) e autor de Pensamento crítico - o poder da lógica e da argumentação, livro escrito em parceria com o matemático americano Richard L. Epstein, explica: “Um argumento é uma ‘viagem lógica’ que vai das premissas à conclusão. Conforme a definição dada no nosso livro, um bom argumento é aquele em que há boas razões para que as premissas sejam verdadeiras, e, para além disso, as premissas apresentam boas razões para suportar ou apoiar a conclusão”.Para Carnielli, os brasileiros têm uma “péssima educação argumentativa”.  
 Confundimos discussão com briga e não sabemos lidar bem com críticas. Mas há técnicas que podem ajudar na construção de bons argumentos - e também a evitar armadilhas comuns em uma discussão, como o uso de falácias. Ainda segundo o autor: “Existe um princípio metodológico importante na argumentação que é o Princípio da Acomodação Racional, também conhecido como Princípio da Caridade, e que foi tratado por filósofos de peso como Willard Van Orman Quine e Donald Davidson. O princípio exige que devemos tentar entender o ponto de vista do oponente em sua forma mais forte e persuasiva antes de submeter sua visão à nossa avaliação. Dessa forma, devemos primeiro fazer todos os esforços para esclarecer as premissas e a conclusão do oponente, inclusive ajudando-o a reparar os pontos fracos. Só então, após essa atitude respeitosa, é que devemos gentilmente apontar a ela ou a ele onde suas premissas são falhas ou duvidosas, e/ou porque tais premissas não apoiam a conclusão. (...) É a maneira mais respeitosa e produtiva de manter uma discussão honesta”.'''
redacao1 = '''Querido filho, 
 Fiquei sabendo que você terá seu primeiro debate público na próxima semana. Eu já conheço bem suas ideias antifeministas e sei que a feminista que debaterá com você é extremista. Sendo seu pai, sinto que é meu dever te lhe dar alguns conselhos. 
 Quando for debater, mantenha a calma e não ataque a argumentadora oponente, mas sim seus argumentos. Esse é o primeiro passo para que o debate seja civilizado e produtivo. Use a razão e a lógica. Caso a feminista faça o oposto disso, e, por exemplo, descarte os seus argumentos meramente por você ser homem, branco, heterossexual e cristão, explique ao público que ela está usando a famosa falácia ad hominem. 
 Evite e aponte, meu querido filho, todas as falácias lógicas. Falácias destroem o bom debate. Caso a feminista se esquive das críticas contra o feminismo, afirmando que aquilo que está sendo criticado por você chama-se “femismo” e não “feminismo”, avise-a que ela está utilizando a falácia do escocês de verdade. 
 Mantenha-se firme também na defesa pela liberdade de expressão. Essa é outra regra de ouro para o bom debate. Não importa o quão verbalmente agressiva seja a feminista ou quão repulsivas pareçam ser suas ideias, lembre-se que ela possui o direito de falar tudo aquilo que realmente pensa. Se tudo o que ela fala for de fato tão horrível e irracional assim, então será fácil para você desconstruir publicamente os argumentos dela. Lembre-se daquilo que sempre te  ensinei: tentar calar o debatedor adversário à força, como se estivesse cometendo um crime por expor o que pensa, é para aqueles que não conseguem convencer os demais através do uso da lógica e da razão, então precisam fazer isso através de meios coercitivos. 
 Portanto, filho, se alguém nesse debate utilizar constantemente argumentos falaciosos ou tentar calar à força o adversário, alegando estar combatendo um “discurso de ódio”, então que esse alguém não seja você. Não podemos controlar a feminista com quem você debaterá, mas você tem controle sobre si próprio. Faça a sua parte e tenha um bom debate! 
 Com carinho, 
 Seu amado Pai.'''

In [20]:
tema3 = '''“Robô-guepardo” do MIT aprendeu a pular, porque a humanidade não conhece limites'''
motivador3= '''A novidade vem do próprio MIT, porque evidentemente é válido a humanidade se gabar das criações que um dia poderão matá-la. No comunicado oficial da instituição, os pesquisadores dizem que “robôs de quatro pernas foram inspirados pelos movimentos de guepardos e outros animais, tendo enorme progresso, mas eles ainda estão bem atrás de seus ‘primos mamíferos’ quando o assunto é atravessar ambientes com rápidas alterações de elevação”.\n“Nessas condições, você precisa usar [um sistema de] visão noturna para evitar uma falha. Por exemplo, transpor um buraco pode ser difícil se o robô não puder vê-lo. Apesar de existirem alguns métodos para incorporar a visão noturna na locomoção por pernas, a maioria não é muito recomendável para sistemas robóticos ágeis”, disse Gabriel Margolis, estudante de Ph.D no Laboratório de Ciências da Computação e Inteligência Artificial (IA) do MIT.'''
redacao3 = '''Google LLC é uma empresa multinacional mano de serviços online e software dos Estados Unidos. O Google hospeda brasil e desenvolve pedro uma série de serviços e produtos baseados na internet e gera lucro principalmente através da publicidade pelo AdWords. A Google é a principal subsidiária da Alphabet Inc.\n  Google LLC é uma empresa multinacional de serviços online e software dos Estados Unidos. O Google hospeda e desenvolve uma série de serviços e produtos baseados na internet e gera lucro principalmente através da publicidade pelo AdWords. A Google é a principal subsidiária da Alphabet Inc.\n  De início, é válido mencionar o impacto parsa negativo da inércia das instituições de ensino.A esse respeito, o expoente educador Paulo Freire,na obra “Pedagogia do Oprimido”,afirma que as escolas devem colaborar para a formação crítica e consciente dos alunos.Entretanto, muitos professores mostram-se indiferentes à proposta de Freire, uma vez que dão enfoque ao repasse de conteúdos das disciplinas e não discutem a importância de cuidar da saúde mental, além da relevância do respeito a indivíduos que possuem algum transtorno na mente.Por conseguinte, muitos discentes sem criticidade e ciência do assunto nutrem estigmas errôneos a respeito desse tema. Assim, não é razoável que o silêncio dos colégios dê lugar a este grave entrave: a discriminação associada às patologias psicológicas no corpo social.'''

In [21]:
tema2 =  '''O que é mais importante para vencer na vida?'''
motivador2 = '''Reportagem publicada pelo UOL Economia no mês passado apresenta uma pesquisa realizada pelo Datafolha em que se perguntava aos entrevistados qual o fator mais importante para se conquistar uma vida melhor. A amostragem reflete toda a população do Brasil, com baixa margem de erro. \nOs resultados da pesquisa revelaram que, em primeiro lugar, os brasileiros consideram necessária a fé religiosa. De acordo com a pesquisa, as pessoas consideram a fé mais importante do que o estudo ou o trabalho, por exemplo, para melhorar de vida. Leia o texto do UOL que se transcreve abaixo, preste atenção nos percentuais e redija uma dissertação argumentativa apresentando sua opinião sobre a questão formulada na pesquisa: para você, dos itens mencionados, qual é o mais importante para melhorar de vida? Por quê? Apresente suas razões para justificar o seu ponto de vista.Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vidaA fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. \n Esse percentual supera até mesmo aqueles que consideram os estudos (21%), o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor.Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição. \n Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%), apoio financeiro da família (5%) e cultura e lazer (2%).Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam:- Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter acesso à aposentadoria: 6% - Apoio financeiro da família: 5% - Cultura e lazer: 2% Pesquisa ouviu 2.086 pessoas em todo o país. \n A Oxfam encomendou a pesquisa ao Instituto Datafolha, que entrevistou pessoalmente 2.086 pessoas em 130 cidades de todos os estados brasileiros. As entrevistas foram feitas entre os dias 12 e 18 de fevereiro. A margem de erro para a amostragem geral é de dois pontos percentuais para mais ou para menos, considerando um nível de confiança de 95%.Por ser uma pesquisa amostral realizada em locais de grande circulação de pessoas, o Datafolha definiu uma amostra que busca refletir o perfil da sociedade brasileira conforme o último Censo, informou a Oxfam.'''
redacao2 = '''Hoje em dia os fatores barra ta limpa mais importantes para vencer na vida é pindaíba focar nos estudos e continuar tentando. No Brasil, esses porras todos nós temos o direito à educação esse  já é o primeiro passo para vencer na vida. Isso evidência não só pela educação e também onde vive. Não haverá futuro para nossos filhos e netos se continuar assim. \n Em diversas cidades do Brasil  as escolas encontram-se em situações precárias beleza, alunos vão a  escola e faltam  professores. esses gays Governadores são corruptos porque homens não prestam e ainda reduzem as verbas da educação. \n Todos têm o direito à educação, mas poucos tem o acesso às escolas, alguns deles por morar em lugares muitos distantes, sem infraestrutura, onde as estradas são de chão, sem iluminação pública biscoitero e os cidadãos não conseguem o acesso o  transporte público porque ele é uma merda. \n Por todos esses aspectos caiu na rede é peixe apresentados conclui-se que essa bicha fresca esses gordos, é  fundamental melhorar o ensino das escolas públicas e melhorar a infraestrutura para facilitar o acesso dos cidadãos as escolas públicas.'''

In [26]:
competencia2.analisar(redacao1,tema1,motivador1)

{'lista_similaridade': ['ideias',
  'caso',
  'debate',
  'debater',
  'discurso',
  'verdade',
  'ódio',
  'fácil',
  'pensa',
  'lógica',
  'críticas',
  'argumentos',
  'falácias',
  'oponente'],
 'predicao_fuga': array([0]),
 'primeira_pessoa': [('eu', [47, 49]), ('me ', [1006, 1009])],
 'resultado': '0.01%',
 'similaridade_semantica': 35,
 'tipo_textual': 'Poesia'}

In [27]:
competencia2.listar_similsent(motivador3)[0]

[('forma', 0.8534658551216125),
 ('maneira', 0.8453959226608276),
 ('tanto', 0.8409250378608704),
 ('assim', 0.8352344036102295),
 ('pois', 0.8344255089759827),
 ('exemplo', 0.8341102004051208),
 ('possível', 0.8301902413368225),
 ('mais', 0.8239567279815674),
 ('mesmo', 0.8231909275054932),
 ('quanto', 0.8225553035736084)]

# **COMPETÊNCIA 3**

Selecionar, relacionar, organizar e interpretar informações, fatos, opiniões e argumentos em defesa de um ponto de vista.

**Última atualização em:** 05/11/2021 por *Felipe Ferreira*.

## Desvios

*   Ausência de Relação Lógica entre as partes do texto;
*   Problema de Precisão vocabular;
*   Falta de progressão temática;
*   Problema de adequação entre o conteúdo do texto e o mundo real;
*   Argumentação Previsível(uso de clichês e Senso Comum) e
*   Argumentação Limitada ao proposto no texto motivador  **✓**
*   Ausência de Posicionamento sobre o tema.

## Documentação

  **|||||||||| Inicialização ||||||||||**

O método construtor desta classe não recebe nenhum argumento.

---
  **|||||||||| Objetivo ||||||||||**

Esta classe estabele com complexo fluxo analítico para uma entrada textual onde os devios contidos nesta competência são computados.


---

 `refCliche:`
*   **Type:** Str ;
*   **Descrição:** Diretório para importar lista com elementos de cliche.

 `refDitados:`
*   **Type:** Str ;
*   **Descrição:** Diretório para importar lista com elementos de ditados populares.

 `refGeneralizacoes:`
*   **Type:** Str ;
*   **Descrição:** Diretório para importar lista com elementos de generalizações.


---


**|||||||||| Execução ||||||||||**

Para executar atribua à classe com os seguintes argumentos:
```
competencia3 = comp3(refCliche,refDitados,refGeneralizacoes)
```
Em seguida atribua o método `analisar(redação,motivador)`, sendo redação sua variável do tipo string a ser analisada e motivador a variável tipo string para comparação.
```
competencia2.analisar(redação,motivador)
```
---

## Classe

In [9]:
class comp3:



  #______Inicialização______#

  
  def importData(self,data): #Subfunction apontada em __init__()
    imp=[]
    with open(os.path.join(data), "r", encoding='utf-8') as f:
      text = f.readlines()
      for palavra in text:
        imp.append(palavra.replace('\n',''))    
    return imp

  def __init__(self, refCliche,refDitados,refGeneralizacoes):
    self.cliche = self.importData(refCliche)
    self.exce = ['a','e','as','o','os','de','da','que','são','é', # Converter para .txt
          'com','do','como','à', 'às','és', 'um','uma','umas',
          'uns','te', 'tem','sem','ser','se','ou','no','na',
          'em','dar', 'mais', 'ao', 'aos', 'bom','cuja',
          'cujo', 'custa', 'cá','das', 'ali','desde',
          'dessa', 'desse', 'desta', 'deste','deve',
          'devem','faz','fez','for', 'fora', 'foram',
          'isso', 'isto','já','lhe','mas', 'nesse',
          'nessa','nesses','nessas','para','em','esse',
          'essa','esses','essas','dos','pois']
    self.pontu = string.punctuation
    self.ditado = self.importData(refDitados)
    self.gener = self.importData(refGeneralizacoes)


  #______Argumentação_Previsível_______#

  def detecCliche(self,txt):
    cliche = []
    ltxt = txt.lower()
    for i in self.cliche:
      var = (i,[ltxt.find(i),ltxt.find(i)+len(i)])
      if ltxt.find(i) != -1:
        cliche.append(var)
    return cliche

  def detecDit(self,txt):
    ditados = []
    ltxt = txt.lower()
    for i in self.ditado:
      var = (i,[ltxt.find(i),ltxt.find(i)+len(i)])
      if ltxt.find(i) != -1:
        ditados.append(var)
    return ditados

  def detecGener(self,txt):
    genera = []
    ltxt = txt.lower()
    for i in self.gener:
      var = (i,[ltxt.find(i),ltxt.find(i)+len(i)])
      if ltxt.find(i) != -1:
        genera.append(var)
    return genera
        
  def argumentPrev(self,txt):
    c = self.detecCliche(txt)
    d = self.detecDit(txt)
    g = self.detecGener(txt)
    argument_cliche_data,argument_dit_data,argument_gener_data = [],[],[]
    for i in c:
      argument_cliche_data.append(i)
    for j in d:
      argument_dit_data.append(j)
    for s in g:
      argument_gener_data.append(s)
    return argument_cliche_data,argument_dit_data,argument_gener_data

  #______END______#

  #______Plágio_do_Texto_motivador______#
  
  def retplag(self,sp_a,sp_m,lis_match,lst_simil):
    plagios,loc = [],[]
    for ind in range(len(lst_simil)):
      if lst_simil[ind] > 0.25: #Fator de similaridade 0.0 ~ 1.0
        if (sp_a[int(re.search(r"[0-9]{1,}$",lis_match[ind]).group())]) not in plagios:
          plagios.append(sp_a[int(re.search(r"[0-9]{1,}$",lis_match[ind]).group())])
        if (sp_m[int(re.search(r"\A[0-9]{1,}",lis_match[ind]).group())]) not in loc:
          loc.append(sp_m[int(re.search(r"\A[0-9]{1,}",lis_match[ind]).group())])
    sempontu=[]
    for fras in plagios:
      frr = fras.replace(',','')
      for u in self.pontu:
        frr = frr.replace(u,'')
      sempontu.append(frr)
    return plagios , loc, sempontu

  def tplag(self,txt,mot):
    var_txt = txt.replace('\n','').replace('...','').replace(',',', ').replace('!','! ').replace('?','? ').replace('  ',' ')
    var_m = mot.replace('\n','').replace('...','').replace(',',', ').replace('!','! ').replace('?','? ').replace('  ',' ')
    sp_a,sp_m = var_txt.split('.'),var_m.split('.')
    for frase in range(len(sp_a)): #remover ' '
      if sp_a[frase] == ' ':
        del(sp_a[frase])
    for frase2 in range(len(sp_m)): #remover ' '
      if sp_m[frase2] == ' ':
        del(sp_m[frase2])
    return sp_a,sp_m

  def new_plag(self,txt,mot):
    sp_a,sp_m = self.tplag(txt,mot)
    lst_simil,lis_match = [],[]
    for f in range(len(sp_m)):
      for a in range(len(sp_a)):
        lis_match.append(f'{f}-{a}') #Marcando momento do laço
        simil, count = 0,0
        spp_a = sp_a[a].split()
        spp_m = sp_m[f].split()
        for palavra in spp_a:
          for pala in spp_m:
            if palavra.lower() not in self.exce:
              if palavra.lower() == pala.lower():
                count+=1
        simil = count/(len(sp_m)+0.00001) #Previne divisão por zero
        lst_simil.append(simil)
    plagi,loc,semp = self.retplag(sp_a,sp_m,lis_match,lst_simil)
    return plagi,loc,semp
  #______END______#

  #______Analise______#
  def analisar(self,txt,mot):
    plagi,loc,semp = self.new_plag(txt,mot)
    cliche_data,dit_data,gener_data = self.argumentPrev(txt)
    data = {'plagios':plagi,
            'plag_loc': loc,
            'sem_pontos': semp,
            'cliches':cliche_data,
            'ditados_pop':dit_data,
            'generalização':gener_data}
    return data

## Execução

In [5]:
refCli = '/content/drive/MyDrive/Spell_AI/Data/bases/cliches.txt'
refGener = '/content/drive/MyDrive/Spell_AI/Data/bases/generalizacoes.txt'
refDit = '/content/drive/MyDrive/Spell_AI/Data/bases/ditadospopulares.txt'
competencia3 = comp3(refCli,refDit,refGener)

In [10]:
competencia3.analisar(txta,txtm)

{'cliches': [],
 'ditados_pop': [],
 'generalização': [('jamais saberemos', [577, 593])],
 'localizacao': [' De acordo com a pesquisa, as pessoas consideram a fé mais importante do que o estudo ou o trabalho, por exemplo, para melhorar de vida',
  'Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vidaA fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil',
  'Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam:- Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter acesso à aposentadoria: 6% - Apoio financeiro da família: 5% - Cultura e lazer: 2% Pesquisa ouviu 2'],
 'plagios': ['A população brasileira apontou a fé como fator importante para melhorar a vida deixando fatores como estudos, acesso à saúde, aposentadoria, trabalho, lazer e outros para trás',


In [ ]:
txta

'A população brasileira apontou a fé como fator importante para melhorar a vida deixando  fatores como estudos, acesso à saúde, aposentadoria, trabalho, lazer e outros para trás. Essa foi a resposta de 28% dos entrevistados  que caracteriza o Brasil como uma sociedade com moral enraizada em preceitos e mandamentos religiosos e que isso gera melhora de vida como consequência. Esse resultado é a coadunação do senso comum com a ciência, pois são numerosas as pesquisas médicas que incluem a fé como assunto a ser tratado em consultórios, hospitais e até grades acadêmicas.\nNós jamais saberemos se a fé colabora para a participação de um fenômeno chamado coping : a capacidade humana de superar adversidades e não é necessário crer em uma divindade, pois os estudos voltados para essa área encontram essa capacidade também em ateus que creem em máximas ou jaculatórias como "vai ficar tudo bem", "eu vou superar isso" e outras afirmações positivas, colhendo efeitos minorados de um diagnóstico de do

In [ ]:
txta.find('DKSOPKDP')

-1

# **COMPETÊNCIA 4**

Demonstrar conhecimento dos mecanismos linguísticos necessários à construção da argumentação.

**Última atualização em:** 27/10/2021 por *Felipe Ferreira*.

## Desvios

*  Frases fragmentadas que comprometem a estrutura lógico-gramatical;
*  Uso repetitivo de conectivos; **✓**
*  Ausência de recursos coesivos;
*  Uso inadequado de conectivos(pronomes relativos, preposições ou conjunções);
*  Repetição ou sustituição inadequada de palavras, sem se valer de recursos como,pronome,advérbio ou artigo.

## Classe

In [ ]:
class comp4:
  def __init__(self):
    self.pontuacoes = string.punctuation
    self.stop_words = pln.Defaults.stop_words

  def preprocessamento(self,text): #Subfunction apontada em palavrasmaisusadas()
    result = []
    texto = re.sub(f'[{string.punctuation}\n]','',text)
    doc = pln(texto.lower())
    for token in doc:
      result.append(token.text)
      texto = ' '.join([str(elemento) for elemento in result if not elemento.isdigit()])         
    return texto

  def palavrasmaisusadas(self,palavras_redacoes):
    sw_l,tag_l=[],[]
    for palavra in palavras_redacoes:
      if palavra in self.stop_words:
        sw_l.append(palavra)
      else:
        tag_l.append(palavra)
    freqSw = nltk.FreqDist(sw_l)
    freqtag = nltk.FreqDist(tag_l)
    maiscomunsSW = freqSw.most_common(5)
    maiscomumTAG = freqtag.most_common(5)    
    return maiscomunsSW,maiscomumTAG

  def analisar(self,entrada):
    limpo = self.preprocessamento(entrada)
    palavras_redacoes = ([palavra for palavra in nltk.word_tokenize(limpo)])
    d1,d2 = self.palavrasmaisusadas(palavras_redacoes)
    d11,d12,d13,d21,d22,d23 =[],[],[],[],[],[]
    for i in range(len(d1)):
      d11.append(d1[i][0])
      d12.append(d1[i][1])
      d13.append(f'{round(d1[i][1]/len(palavras_redacoes)*100,2)}%')
      d21.append(d2[i][0])
      d22.append(d2[i][1])
      d23.append(f'{round(d2[i][1]/len(palavras_redacoes)*100,2)}%')
    data = {'Stop_word_lis':d11,
            'Stop_word_freq':d12,
            'Stop_word_freqn':d13,
            'Outras_lis':d21,
            'Outras_freq':d22,
            'Outras_freqn':d23}
    return data 

## Execução

In [ ]:
competencia4 = comp4()

In [ ]:
competencia4.analisar(txta)

{'Outras_freq': [10, 4, 3, 2, 2],
 'Outras_freqn': ['3.22%', '1.29%', '0.96%', '0.64%', '0.64%'],
 'Outras_lis': ['fé', 'vida', 'estudos', 'população', 'apontou'],
 'Stop_word_freq': [21, 10, 9, 9, 8],
 'Stop_word_freqn': ['6.75%', '3.22%', '2.89%', '2.89%', '2.57%'],
 'Stop_word_lis': ['a', 'que', 'e', 'de', 'em']}

# **COMPETÊNCIA 5**

Elaborar proposta de Intervenção para o problema abordado, respeitando os Direitos Humanos

**Última atualização em:** 27/10/2021 por *Felipe Ferreira*.

## Desvios

* Proposta desrespeitou os direitos humanos. Anula redação **✓**
* Intervenção não detalhada
* Intervenção não articulada com a discussão desenvolvida
* Intervenção vaga ou precária
* Intervenção relacionada apenas ao assunto
* Ausência da Proposta de Intervenção

## Classe

In [ ]:
class comp5:

  def importData(self,data): #Subfunction apontada em __init__()
    dat = open(data,'r').readlines()
    imp = []
    for j in dat:
      varj = j.replace('\n','')
      imp.append(varj)
    return imp
  
  def __init__(self,refPrecon,refBaixoC):
    self.precon = self.importData(refPrecon)
    self.baixoc = self.importData(refBaixoC)
  
  #______DireitosHumanos______#
  def detecPrecon(self,entrada): #Detecção de preconceito.
    lis_precon = []        
    for palavra in self.precon:
      if palavra in entrada:
        lis_precon.append(palavra)
    return lis_precon
  
  def detecBaixoC(self,entrada): #Detecção de palavras de
    lis_baixo=[]                 # baixo calão.
    for palavra in self.baixoc:
      if palavra in entrada:
        lis_baixo.append(palavra)
    return lis_baixo
  #______END______#

  #______Execução______#
  
  def analisar(self,entrada): #Função de análise textual.
    lis_precon = self.detecPrecon(entrada)
    lis_baixo = self.detecBaixoC(entrada)
    data = {'preconceitos':lis_precon,
            'baixo_calao':lis_baixo
            }

    return data

## Execução

In [ ]:
preconceitos = '/content/drive/MyDrive/Spell_AI/Data/vocabularios/preconceitos.txt'
baixocalao= '/content/drive/MyDrive/Spell_AI/Data/vocabularios/baixoc.txt'

In [ ]:
competencia5 = comp5(preconceitos,baixocalao)

In [ ]:
competencia5.analisar(ent)

{'baixo_calao': [], 'preconceitos': []}

# Funções Suplementares

Task extras às contidas nas competências do ENEM.

**Última atualização em:** 05/11/2021 por Felipe Ferreira.

## Documentação

**|||||||||| Objetivo ||||||||||**

Ser uma classe repositório onde podemos instruir funções extras. 

---

**|||||||||| Inicialização ||||||||||**

O método construtor desta classe recebe `NENHUM` argumento.


---


**|||||||||| Execução ||||||||||**

Para executar atribua à classe a uma variável.
```
suplementares = suplement()
```
Em seguida atribua o método desejado `.metodo(redação)`, sendo redação sua variável do tipo string a ser analisada.
```
suplementares.metodo(redação)
```


---



## Classe

In [ ]:
class suplement:
  def __init__(self):
    return

  def linhas(self,txt):
    linhas = int(len(txt)/72)
    return linhas

  def paragrafos(self,txt):
    var = txt.split('\n')
    paragr = len(var)
    return paragr
  
  def analisar(self,txt):
    l = self.linhas(txt)
    p = self.paragrafos(txt)
    data = {'linhas':l,
            'paragrafos':p}
    return data

## Execução

In [ ]:
suplementar = suplement()

In [ ]:
suplementar.analisar(txta)

{'linhas': 26, 'paragrafos': 4}

# Execução ao longo de um DataFrame

## Preparação

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Spell_AI/IARedaçãoUnica_edit - Base NF_800.csv')

In [ ]:
df.head()

,created_at,slug,user_id,token,corrected_essay,redacao,tema,motivador,grades,link,nota_1,nota_2,nota_3,nota_4,nota_5,nota_final,len(redacao),/n_fotoredacao,linhasfoto
0,2021-07-02T13:36:52,enem-e-vestibulares/redacao/redp-propostas-de-...,4339537,Ntj4YVWbfe3UVqKdzwUTPA,NaN,"O escritor modernista, Carlos Drummond de Andr...",A identidade brasileira entre a diversidade cu...,“Nós somos um país paradoxal. Ora o Brasil sur...,"{""grade_1"": 120, ""grade_2"": 200, ""grade_3"": 20...",https://cdn.mesalva.com/uploads/essay_submissi...,120,200,200,200,200,920,2518,4,30
1,2021-07-02T13:34:01,enem-e-vestibulares/redacao/redp-propostas-de-...,4107966,vJr5wm_bdYtcU7rhOl7RUA,NaN,Em 1500 começou a troca de cultura de forma nã...,A identidade brasileira entre a diversidade cu...,“Nós somos um país paradoxal. Ora o Brasil sur...,"{""grade_1"": 80, ""grade_2"": 160, ""grade_3"": 160...",https://cdn.mesalva.com/uploads/essay_submissi...,80,160,160,200,200,800,2151,3,27
2,2021-06-29T01:29:30,enem-e-vestibulares/redacao/redp-propostas-de-...,3097053,3hUytKRu-2yDBrLJIrY30Q,NaN,"""Moro num país tropical, abençoado por Deus e ...",A identidade brasileira entre a diversidade cu...,“Nós somos um país paradoxal. Ora o Brasil sur...,"{""grade_1"": 160, ""grade_2"": 200, ""grade_3"": 20...",https://cdn.mesalva.com/uploads/essay_submissi...,160,200,200,200,200,960,2214,3,28
3,2021-06-28T17:18:31,enem-e-vestibulares/redacao/redp-propostas-de-...,3731542,QHgjiSztIFQwfeNUN78eNw,NaN,"Desde a Ditadura Militar de 1964, houve uma de...",A identidade brasileira entre a diversidade cu...,“Nós somos um país paradoxal. Ora o Brasil sur...,"{""grade_1"": 160, ""grade_2"": 200, ""grade_3"": 16...",https://cdn.mesalva.com/uploads/essay_submissi...,160,200,160,160,200,880,1683,4,29
4,2021-06-30T15:58:33,enem-e-vestibulares/redacao/redp-propostas-de-...,1631111,Soc9E7pp0gCqkuHbA-LmpA,NaN,A Semana da Arte Moderna foi um aveto no qual ...,A identidade brasileira entre a diversidade cu...,“Nós somos um país paradoxal. Ora o Brasil sur...,"{""grade_1"": 200, ""grade_2"": 200, ""grade_3"": 16...",https://cdn.mesalva.com/uploads/essay_submissi...,200,200,160,200,200,960,2910,4,30


In [ ]:
print(df['nota_final'].tolist())

[920, 800, 960, 880, 960, 920, 960, 880, 880, 840, 960, 960, 960, 880, 880, 960, 920, 880, 920, 880, 960, 920, 960, 960, 840, 920, 960, 920, 960, 880, 960, 840, 880, 840, 960, 800, 920, 960, 920, 920, 920, 960, 920, 880, 960, 920, 960, 800, 920, 960, 880, 840, 840, 840, 920, 920, 960, 840, 960, 960, 960, 920, 920, 960, 920, 960, 920, 880, 840, 920, 920, 960, 920, 880, 920, 920, 840, 800, 880, 960, 880, 920, 920, 960, 840, 880, 800, 880, 840, 800, 960, 880, 920, 920, 840, 800, 920, 960, 840, 920, 960, 840, 800, 920, 920, 960, 840, 920, 800, 840, 960, 920, 880, 800, 960, 880, 960, 800, 920, 840, 960, 800, 960, 800, 880, 880, 880, 800, 920, 960, 800, 800, 880, 840, 880, 960, 840, 800, 880, 920, 800, 800, 920, 920, 840, 920, 960, 920, 800, 920, 880, 880, 920, 880, 920, 840, 880, 960, 960, 960, 960, 840, 840, 880, 800, 920, 920, 800, 960, 920, 1000, 920, 800, 880, 920, 880, 800, 880, 920, 960]


In [ ]:
red_list = df['redacao'].tolist()
tem_list = df['tema'].tolist()
mot_list = df['motivador'].tolist()

In [ ]:
red_list

['O escritor modernista, Carlos Drummond de Andrade, em seu poema "No meio do caminho", retrata, de modo figurado, os contratempos que o ser humano sofre em sua jornada. Nesse viés, pode-se assimilar que a temática da identidade brasileira entre a diversidade cultural e a precariedade social é um grande desafio presente na trilha de um país em desenvolvimento ocasionado, sobretudo, pela desigualdade econômica populacional, somado ao preconceito cultural. Logo, faz-se necessário compreender esse entrave para conorná-lo.\r\nA princípio, vale ressaltar a desigualdade econômica das calsses sociais no revés. A título de exemplo, uma pesquisa realizada pelo Instituto Brasileiro de Geografia e Estatística (IBGE), mostra que os índices de violência entre os jovens aumentou cerca de 12% entre os anos de 2017 a 2019. Nesse sentido, é possível observar que apesar dos inúmeros recursos naturais, a agricultura e a pecuária refletindo uma economia de alto potencial, não garante a qualidade de vida e

## Resultados

In [ ]:
f' Total de redações a serem analisadas é de {len(red_list)} redações tendo {len(tem_list)} temas e {len(mot_list)} textos motivadores'

' Total de redações a serem analisadas é de 180 redações tendo 180 temas e 180 textos motivadores'

In [ ]:
r1_l,r2_l,r3_l,r4_l,r5_l,rs_l = [],[],[],[],[],[]
for i in tqdm(range(len(red_list))): #Executa análise de cada competênca em cada redação contida no Df.
  r1_l.append(competencia1.analisar(red_list[i]))
  r2_l.append(competencia2.analisar(red_list[i],tem_list[i],mot_list[i]))
  r3_l.append(competencia3.analisar(red_list[i],mot_list[i]))
  r4_l.append(competencia4.analisar(red_list[i]))
  r5_l.append(competencia5.analisar(red_list[i]))
  rs_l.append(suplementar.analisar(red_list[i]))

100%|██████████| 180/180 [11:22<00:00,  3.79s/it]


In [ ]:
r1_l

[{'gerundismo': ([], []),
  'girias': ['mano'],
  'ortografia': ['conorná-lo', 'calsses', 'diversar', 'aligia'],
  'posi_virgula': [(188, 193)],
  'uso_de_virgula': ['assim'],
  'voz_passiva': [' pode-se assimilar que a temática da identidade brasileira entre a diversidade cultural e a precariedade social é um grande desafio presente na trilha de um país em desenvolvimento ocasionado',
   ' faz-se necessário compreender esse entrave para conorná-lo',
   ' observa-se que apesar da mistura de povos',
   '\rInfere-se']},
 {'gerundismo': ([], []),
  'girias': ['mina'],
  'ortografia': ['portuguêse',
   'mestisso',
   'etínicos',
   'pocoaria',
   'potecial',
   'caius',
   'sóciais',
   'envestimentos',
   'aumentosdos',
   'indices',
   'igualitario'],
  'posi_virgula': [(1654, 1662), (1144, 1149), (1005, 1021), (1160, 1166)],
  'uso_de_virgula': ['portanto', 'assim', 'consequentemente', 'porque'],
  'voz_passiva': [' de modo que os portuguêse e europeus na exploração trazia escravos afri

In [ ]:
r2_l

[{'lista_similaridade': ['país',
   'brasil',
   'violência',
   'recursos',
   'naturais',
   'agricultura',
   'pecuária',
   'potencial',
   'economia',
   'sentido',
   'cultura'],
  'predicao_fuga': array([0]),
  'primeira_pessoa': [('eu', [55, 57]), ('me ', [1092, 1095])],
  'resultado': '0.28%',
  'similaridade_semantica': 15,
  'tipo_textual': 'Dissertação ENEM'},
 {'lista_similaridade': ['país',
   'brasil',
   'exploração',
   'mundo',
   'agricultura',
   'crescimento',
   'economia',
   'cultura',
   'pública'],
  'predicao_fuga': array([1]),
  'primeira_pessoa': [('eu', [106, 108])],
  'resultado': '99.92%',
  'similaridade_semantica': 18,
  'tipo_textual': 'Dissertação ENEM'},
 {'lista_similaridade': ['país',
   'brasil',
   'direitos',
   'natureza',
   'própria',
   'cultura',
   'sociedade',
   'criação'],
  'predicao_fuga': array([0]),
  'primeira_pessoa': [('eu', [40, 42])],
  'resultado': '1.59%',
  'similaridade_semantica': 17,
  'tipo_textual': 'Dissertação ENEM'}

In [ ]:
r3_l

[{'argumentacao_previsivel': [],
  'localizacao': [' Ora o Brasil surge como uma região exótica, de praias paradisíacas, florestas edênicas, carnaval, capoeira e futebol; ora como um lugar execrável, de violência urbana, exploração da prostituição infantil, desrespeito aos direitos humanos e desdém pela natureza',
   ' Ora festejado como um dos países mais bem preparados para ocupar o lugar de protagonista no mundo – amplos recursos naturais, agricultura, pecuária e indústria diversificadas, enorme potencial de crescimento de produção e consumo; ora destinado a um eterno papel acessório, de fornecedor de matéria-prima e produtos fabricados com mão de obra barata, por falta de competência para gerir a própria riqueza',
   ' E permanecemos em terceiro lugar entre os mais desiguais entre todos \r\rVolto, então, à pergunta inicial: o que significa habitar essa região situada na periferia do mundo, escrever em português para leitores quase inexistentes, lutar, enfim, todos os dias, para con

In [ ]:
r4_l

[{'Outras_freq': [4, 4, 4, 2, 2],
  'Outras_freqn': ['1.04%', '1.04%', '1.04%', '0.52%', '0.52%'],
  'Outras_lis': ['cultural', 'país', 'preconceito', 'drummond', 'identidade'],
  'Stop_word_freq': [23, 23, 14, 10, 9],
  'Stop_word_freqn': ['5.99%', '5.99%', '3.65%', '2.6%', '2.34%'],
  'Stop_word_lis': ['de', 'a', 'o', 'que', 'e']},
 {'Outras_freq': [6, 6, 5, 3, 2],
  'Outras_freqn': ['1.72%', '1.72%', '1.44%', '0.86%', '0.57%'],
  'Outras_lis': ['brasil',
   'desigualdade',
   'educação',
   'desenvolvimento',
   'troca'],
  'Stop_word_freq': [19, 17, 15, 14, 10],
  'Stop_word_freqn': ['5.46%', '4.89%', '4.31%', '4.02%', '2.87%'],
  'Stop_word_lis': ['de', 'o', 'a', 'que', 'e']},
 {'Outras_freq': [6, 5, 5, 4, 4],
  'Outras_freqn': ['1.81%', '1.51%', '1.51%', '1.2%', '1.2%'],
  'Outras_lis': ['identidade',
   'país',
   'cultura',
   'desvalorização',
   'desigualdade'],
  'Stop_word_freq': [20, 12, 12, 10, 7],
  'Stop_word_freqn': ['6.02%', '3.61%', '3.61%', '3.01%', '2.11%'],
  'Sto

In [ ]:
r5_l

[{'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': ['puta'], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': ['puta'], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': ['puta'], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao': [], 'preconceitos': []},
 {'baixo_calao':

In [ ]:
lis_ort=[]
for i in range(len(r1_l)):
  var = r1_l[i]
  for palavra in var['ortografia']: #Retira todas as palavras identificadas como erradas.
    if palavra not in lis_ort:
      lis_ort.append(palavra)

In [ ]:
len(lis_ort)

454

In [ ]:
lis_ort

['conorná-lo',
 'calsses',
 'diversar',
 'aligia',
 'portuguêse',
 'mestisso',
 'etínicos',
 'pocoaria',
 'potecial',
 'caius',
 'sóciais',
 'envestimentos',
 'aumentosdos',
 'indices',
 'igualitario',
 'braileira',
 'contumam',
 'espetáculos-abertos',
 'público-com',
 'cmo',
 'Cachoeria',
 '-',
 'cultral',
 'iniciativaz',
 'cranças',
 'aveto',
 'artitas',
 'entretantp',
 'identidde',
 'eletizada',
 'propoções',
 'caracterísricas',
 'esterior',
 'grabnde',
 'socioeconônimas',
 'Cidadaia',
 'estrelasNossas',
 'floresNossos',
 'vidaNossa',
 'precesso',
 'importânciado',
 'n',
 'colonizadoes',
 'astística',
 'comprtamental',
 'inacetável',
 'protagoista',
 'indio',
 'cominidades',
 'eivindicou',
 'mandator',
 'ssênciais',
 'olofotes',
 'magnetude',
 'iderário',
 'braileiro',
 'precaridade',
 'sentimeto',
 'possivo',
 'apioar',
 'abrandono',
 'escrassez',
 'latinfúndios',
 'não-homogeinedade',
 'sociologo',
 'oapises',
 'contrução',
 'alsofia',
 'escode',
 'historicamentena',
 'relflexão',

In [ ]:
lis_vp=[]
for i in range(len(r1_l)):
  var = r1_l[i]
  for palavra in var['voz_passiva']: #Retira todas as frases identificadas como contendo uso da voz passiva.
    if palavra not in lis_vp:
      lis_vp.append(palavra)

In [ ]:
len(lis_vp)

898

In [ ]:
lis_vp

[' pode-se assimilar que a temática da identidade brasileira entre a diversidade cultural e a precariedade social é um grande desafio presente na trilha de um país em desenvolvimento ocasionado',
 ' faz-se necessário compreender esse entrave para conorná-lo',
 ' observa-se que apesar da mistura de povos',
 '\rInfere-se',
 ' de modo que os portuguêse e europeus na exploração trazia escravos africanos e o povo europeu se tornara cada vez mais frequente naquele período já existia a desigualdade de forma explícita',
 ' vêmos que a desigualdade no Brasil continua por varios motivos de maneira que o povo mestisso que aqui se faz presente',
 ' ele afirma uma alta relevância dos costumes estrangeiros em detrimento da cultura nativa',
 ' tem-se a valorização de cinematografias e músicas estrangeiras',
 ' pois essas contumam fazer referências aos países onde foram criadas',
 ' visto que nem todas as culturas são igualmente respeitadas',
 ' que em muitos casos são classificadas como vulgares por 

In [ ]:
lis_r4l,var=[],[]
for i in range(len(r4_l)):
  var.append(r4_l[i])
  for palavra in var[i]['Stop_word_freqn']: #Retira todas as palavras identificadas como erradas.
    if palavra not in lis_r4l:
      lis_r4l.append(palavra)

In [ ]:
lis_r4l

['5.99%',
 '3.65%',
 '2.6%',
 '2.34%',
 '5.46%',
 '4.89%',
 '4.31%',
 '4.02%',
 '2.87%',
 '6.02%',
 '3.61%',
 '3.01%',
 '2.11%',
 '6.77%',
 '3.59%',
 '2.79%',
 '2.39%',
 '5.31%',
 '4.67%',
 '3.18%',
 '7.46%',
 '3.31%',
 '2.76%',
 '2.21%',
 '1.93%',
 '4.99%',
 '4.76%',
 '3.4%',
 '2.95%',
 '2.49%',
 '5.5%',
 '4.97%',
 '3.14%',
 '2.62%',
 '4.78%',
 '2.81%',
 '2.53%',
 '2.25%',
 '4.34%',
 '4.08%',
 '3.83%',
 '3.57%',
 '2.55%',
 '4.28%',
 '3.29%',
 '2.3%',
 '5.92%',
 '4.65%',
 '2.75%',
 '2.54%',
 '2.33%',
 '4.72%',
 '3.54%',
 '4.75%',
 '3.48%',
 '2.85%',
 '5.03%',
 '4.19%',
 '3.63%',
 '2.23%',
 '5.86%',
 '3.91%',
 '3.26%',
 '2.61%',
 '2.44%',
 '2.09%',
 '5.41%',
 '5.1%',
 '3.5%',
 '5.2%',
 '4.05%',
 '2.89%',
 '2.31%',
 '3.77%',
 '3.23%',
 '2.96%',
 '2.7%',
 '4.94%',
 '4.56%',
 '3.42%',
 '3.04%',
 '2.66%',
 '4.64%',
 '2.14%',
 '1.79%',
 '6.46%',
 '5.17%',
 '3.88%',
 '2.58%',
 '6.21%',
 '4.47%',
 '3.3%',
 '2.72%',
 '1.75%',
 '6.3%',
 '5.04%',
 '3.36%',
 '2.94%',
 '1.68%',
 '5.47%',
 '4.18%',


In [ ]:
rs_l

[{'linhas': 34, 'paragrafos': 4},
 {'linhas': 29, 'paragrafos': 3},
 {'linhas': 30, 'paragrafos': 4},
 {'linhas': 23, 'paragrafos': 4},
 {'linhas': 40, 'paragrafos': 4},
 {'linhas': 33, 'paragrafos': 5},
 {'linhas': 38, 'paragrafos': 4},
 {'linhas': 33, 'paragrafos': 4},
 {'linhas': 32, 'paragrafos': 4},
 {'linhas': 33, 'paragrafos': 4},
 {'linhas': 27, 'paragrafos': 4},
 {'linhas': 42, 'paragrafos': 4},
 {'linhas': 30, 'paragrafos': 4},
 {'linhas': 30, 'paragrafos': 3},
 {'linhas': 31, 'paragrafos': 4},
 {'linhas': 29, 'paragrafos': 4},
 {'linhas': 25, 'paragrafos': 4},
 {'linhas': 30, 'paragrafos': 4},
 {'linhas': 31, 'paragrafos': 4},
 {'linhas': 33, 'paragrafos': 4},
 {'linhas': 24, 'paragrafos': 4},
 {'linhas': 26, 'paragrafos': 4},
 {'linhas': 33, 'paragrafos': 4},
 {'linhas': 48, 'paragrafos': 4},
 {'linhas': 21, 'paragrafos': 4},
 {'linhas': 28, 'paragrafos': 4},
 {'linhas': 42, 'paragrafos': 4},
 {'linhas': 30, 'paragrafos': 4},
 {'linhas': 30, 'paragrafos': 4},
 {'linhas': 23